In [2]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input/project-data")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [14]:
library(diplyr)
library(ggplot2)
library(lattice)
library(lme4)
library(caret)
library(aod)
library(DT)
library(rstanarm)
library(broom)
library(arm)
library(car)
library(corrplot)

In [40]:
participant_data <- read_csv("../input/project-data/participant_data.csv", show_col_types = FALSE)
ema_data <- read_csv("../input/project-data/p_data_lr.csv", show_col_types = FALSE)

In [41]:
ema_data$ema <- factor(ema_data$ema,levels=c(0,1))

In [42]:
par(mfrow=c(1,3))
for(i in 1:3) {
    boxplot(ema_data[,i], main=names(ema_data)[i])
}

In [47]:
target_df <- data.frame(table(ema_data$ema))
colnames(target_df) <- c("target", "freq")
ggplot(data=target_df, aes(x=target, y=freq, fill=target)) +
  geom_bar(position = 'dodge', stat='identity', alpha=0.5) +
  scale_fill_manual("legend", values = c("1" = "dodgerblue", "0"="firebrick1")) +
  theme_classic()

In [48]:
col_names <- colnames(ema_data)
temp <- gather(ema_data[,col_names, with=F], key="features", value="value", -ema)
temp$ema <- factor(temp$ema)
temp$features <- factor(temp$features, levels=col_names, labels=col_names)
ggplot(data=temp, aes(x=value)) +
  geom_density(aes(fill=ema, color=ema), alpha=0.3) +
  scale_color_manual(values = c("Micro-EMA" = "dodgerblue", "Traditional-EMA"="firebrick1")) +
  theme_classic() +
  facet_wrap(~ features, ncol = 3, scales = "free")
ggplot(data=temp, aes(y=value)) +
  geom_boxplot(aes(fill=ema, color=ema), alpha=0.3) +
  scale_color_manual(values = c("Micro-EMA" = "dodgerblue", "Traditional-EMA"="firebrick1")) +
  theme_classic() +
  facet_wrap(~ features, ncol = 4, scales = "free")

In [16]:
correlations <- cor(ema_data[,1:3])
corrplot::corrplot(correlations, method = 'color')

In [17]:
pairs(ema_data, col=ema_data$ema)

Not much correlation between the predictors

In [38]:
mydata <- ema_data %>%
  dplyr::select_if(is.numeric) 
predictors <- colnames(mydata)
# Bind the logit and tidying the data for plot
mydata <- mydata %>%
  mutate(logit = log(probabilities/(1-probabilities))) %>%
  gather(key = "predictors", value = "predictor.value", -logit)

In [39]:
ggplot(mydata, aes(logit, predictor.value))+
  geom_point(size = 0.5, alpha = 0.5) +
  geom_smooth(method = "loess") + 
  theme_bw() + 
  facet_wrap(~predictors, scales = "free_y")

Assumption of linearity with logit function still holds.

In [ ]:
ema_data$ema <- factor(ema_data$ema,levels=c(0,1), labels=c("Traditional-EMA","Micro-EMA"))

In [18]:
fit <- glm(ema ~ answers + prompts + sleep_hours, data=ema_data, family=binomial())
summary(fit)

Larger P values, means fail to reject null hyp

In [19]:
x <- model.matrix(ema ~ . - 1, data = ema_data)
y <- ema_data$ema
options(mc.cores = parallel::detectCores())
t_prior <- student_t(df = 7, location = 0, scale = 2.5)
post1 <- stan_glm(ema ~ ., data = ema_data,
                 family = binomial(link = "logit"), 
                 prior = t_prior, prior_intercept = t_prior, QR=TRUE,
                 seed = 14124869)
library(ggplot2)
pplot<-plot(post1, "areas", prob = 0.95, prob_outer = 1)
pplot+ geom_vline(xintercept = 0)

posterior distribution for the parameters describing the uncertainty related to unknown parameter values

In [20]:
exp(cbind(OR = coef(fit), confint(fit)))

In [21]:
wald.test(b = coef(fit), Sigma = vcov(fit), L = cbind( 0, 0.5, 0.5, -1))

No effect of predictors

In [23]:
deviance(fit)/df.residual(fit) 

No overdispersion

In [24]:
fit.od <- glm(ema ~ answers + prompts + sleep_hours, data=ema_data, family = quasibinomial())
pchisq(summary(fit.od)$dispersion * fit$df.residual, fit$df.residual, lower = F)

p value is not significant, hence overdispersion is not a problem in our model

In [25]:
with(fit, null.deviance - deviance)
with(fit, pchisq(null.deviance - deviance, df.null - df.residual, lower.tail = FALSE))

In [26]:
ema_data %>%
  mutate(prob = ifelse(ema == "Micro-EMA", 1, 0)) %>%
  ggplot(aes(sleep_hours, prob)) +
  geom_point(alpha = 0.2) +
  geom_smooth(method = "glm", method.args = list(family = "binomial")) +
  labs(
    x = "Sleep Hours",
    y = "Probability of being Micro-EMA"
    )
ema_data %>%
  mutate(prob = ifelse(ema == "Micro-EMA", 1, 0)) %>%
  ggplot(aes(answers, prob)) +
  geom_point(alpha = 0.2) +
  geom_smooth(method = "glm", method.args = list(family = "binomial")) +
  labs(
    x = "Prompts Answered",
    y = "Probability of being Micro-EMA"
    )
ema_data %>%
  mutate(prob = ifelse(ema == "Micro-EMA", 1, 0)) %>%
  ggplot(aes(prompts, prob)) +
  geom_point(alpha = 0.2) +
  geom_smooth(method = "glm", method.args = list(family = "binomial")) +
  labs(
    x = "Prompts Shown",
    y = "Probability of being Micro-EMA"
    )

In [27]:
model_glm_pred = ifelse(predict(fit, type = "link") > 0, "Micro-EMA", "Traditional-EMA")

In [28]:
probabilities <- predict(fit, type = "response")
predicted.classes <- ifelse(probabilities > 0.5, "Micro-EMA", "Traditional-EMA")

In [31]:
plot(fit, which = 4, id.n = 3)

Influential values are extreme individual data points that can alter the quality of the logistic regression model.

The most extreme values in the data can be examined by visualizing the Cook’s distance values. o check whether the data contains potential influential observations, the standardized residual error can be inspected. Data points with an absolute standardized residuals above 3 represent possible outliers and may deserve closer attention.

In [32]:
model.data <- augment(fit) %>% 
  mutate(index = 1:n()) 
model.data %>% top_n(3, .cooksd)
ggplot(model.data, aes(index, .std.resid)) + 
  geom_point(aes(color = ema), alpha = .5) +
  theme_bw()

There is no influential observations in our data.

In [33]:
vif(fit)

Multicollinearity corresponds to a situation where the data contain highly correlated predictor variables. Read more in Chapter @ref(multicollinearity).

Multicollinearity is an important issue in regression analysis and should be fixed by removing the concerned variables.

As a rule of thumb, a VIF value that exceeds 5 or 10 indicates a problematic amount of collinearity. there is no collinearity: all variables have a value of VIF well below 5.

In [34]:
plot(fit, which=1)

Here, the mean of residuals is roughly 0, hence our linearity assumption 

In [35]:
binnedplot(fitted(fit), 
           residuals(fit, type = "response"), 
           nclass = NULL, 
           xlab = "Expected Values", 
           ylab = "Average residual", 
           main = "Binned residual plot", 
           cex.pts = 0.8, 
           col.pts = 1, 
           col.int = "gray")

Hosmer and Lemeshow have proposed a goodness of fit for logistic regression models that can be used with individual data. The basic idea is to create groups using predicted probabilities, and then compare observed and fitted counts of successes and failures on those groups using a chi-squared statistic. Simulation has shown that with g groups the large sample distribution of the test statistic is approximately chi-squared with g-2 degrees of freedom.

In [36]:
hosmer <- function(y, fv, groups=10, table=TRUE, type=2) {
q <- quantile(fv, seq(0,1,1/groups), type=type)
fv.g <- cut(fv, breaks=q, include.lowest=TRUE)
obs <- xtabs( ~ fv.g + y)
fit <- cbind( e.0 = tapply(1-fv, fv.g, sum), e.1 = tapply(fv, fv.g, sum))
if(table) print(cbind(obs,fit))  
chi2 <- sum((obs-fit)^2/fit)
pval <- pchisq(chi2, groups-2, lower.tail=FALSE)
data.frame(test="Hosmer-Lemeshow",groups=groups,chi.sq=chi2,pvalue=pval)
}

In [37]:
hosmer(ema_data$ema, fitted(fit))

We get a Hosmer-Lemeshow chi-squared value of 5.74 on 2 d.f. with a p-value of 0.676, and thus no evidence of lack of fit.